<h3><b>Train and evaluate a chatbot based on an encoder-decoder transformer model ( i.e. same as the original transformer model ) . The model is trained on the Cornell-Movie-Dialog dataset.</b></h3>

<h5><b> 0. Setup</b></h5>

In [6]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from transformer_model import PositionalEncoding, MultiHeadAttentionLayer
from dataset import DatasetHp, preprocess_sentence, get_cornell_dataset
from transformer_model import ModelHp, encoder_decoder_transformer

<h5><b> 1. Load dataset and tokenizer</b></h5>

In [3]:
dataset_hp = DatasetHp(
    max_length = 40,
    vocab_size = 10_000,
    max_sample=50_000,
)

dataset, tokenizer = get_cornell_dataset(dataset_hp)

9916637/9916637 [==============================] - 2s 0us/step
loading conversations ... 


 22%|██▏       | 18638/83097 [00:02<00:09, 6920.18it/s]


initializing tokenizer ...
tokenizer saved in `./transformer/tokenizer`
vocab size updated from 10000 to 10255
tokenization ... 


50000it [00:02, 19004.71it/s]


<h5><b> 2. Define loss and metric functions.</b></h5>

In [4]:
optimizer = tf.keras.optimizers.Adam()

cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction="none"
)

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, dataset_hp.max_length - 1))
    loss = cross_entropy(y_true, y_pred)
    mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    loss = tf.multiply(loss, mask)
    return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, dataset_hp.max_length - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

<h5><b> 3. Build and train the model.</b></h5>

In [7]:
hparams = ModelHp(
    d_model = 256,
    num_attention_heads = 8,
    dropout_rate = 0.1,
    num_units = 512,
    activation = "relu",
    vocab_size = 10255,
    num_layers = 2,
)

#model = encoder_decoder_transformer(hparams, "transformer")

model = tf.keras.models.load_model(
    "model_checkpoint_cb.h5",
    custom_objects={"PositionalEncoding":PositionalEncoding,
                    "MultiHeadAttentionLayer": MultiHeadAttentionLayer},
    compile= False)
print(f"Total number of model's parameters: {model.count_params()}")

Total number of model's parameters: 10521871


In [8]:
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model_checkpoint_cb.h5")

In [10]:
history = model.fit(dataset, epochs=50, callbacks=[model_checkpoint_cb])

Epoch 1/50
1382/1382 [==============================] - 62s 45ms/step - loss: 0.4689 - accuracy: 0.2060
Epoch 2/50
1382/1382 [==============================] - 63s 46ms/step - loss: 0.4680 - accuracy: 0.2060
Epoch 3/50
1382/1382 [==============================] - 64s 46ms/step - loss: 0.4666 - accuracy: 0.2062
Epoch 4/50
1382/1382 [==============================] - 62s 45ms/step - loss: 0.4664 - accuracy: 0.2064
Epoch 5/50
1382/1382 [==============================] - 62s 45ms/step - loss: 0.4660 - accuracy: 0.2065
Epoch 6/50
1382/1382 [==============================] - 63s 45ms/step - loss: 0.4649 - accuracy: 0.2066
Epoch 7/50
1382/1382 [==============================] - 61s 44ms/step - loss: 0.4641 - accuracy: 0.2071
Epoch 8/50
1382/1382 [==============================] - 63s 45ms/step - loss: 0.4632 - accuracy: 0.2073
Epoch 9/50
1382/1382 [==============================] - 63s 46ms/step - loss: 0.4626 - accuracy: 0.2072
Epoch 10/50
1382/1382 [==============================] - 62s 45m

In [11]:
def inference(hp, model, tokenizer, sentence):
    sentence = preprocess_sentence(sentence)
    sentence = tf.expand_dims(
        hp.start_token + tokenizer.encode(sentence) + hp.end_token, axis=0)
    
    output = tf.expand_dims(hp.start_token, 0)

    for i in range(hp.max_length):
        predictions = model(inputs=[sentence, output], training=False)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if tf.equal(predicted_id, hp.end_token[0]):
            break
        
        output = tf.concat([output, predicted_id], axis=-1)
    
    return tf.squeeze(output, axis=0)

def generate_response(hp, model, tokenizer, sentence):
    prediction = inference(hp, model, tokenizer, sentence)
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size]
    )
    return predicted_sentence

def evaluate(hp, model, tokenizer, inputs):
    print("-evaluating ...")
    response = "what are you going to do?"

    for user_sentnece in inputs:
        if user_sentnece != None:
            print(f"\nInput: {user_sentnece}")
            response = generate_response(hp, model, tokenizer, user_sentnece)
            print(f"Output: {response}")
        
        else:
            print(f"\nInput: {response}")
            response = generate_response(hp, model, tokenizer, response)
            print(f"Output: {response}")

In [12]:
from dataset import load_tokenizer

tokenizer = load_tokenizer("./transformer/tokenizer")

In [13]:
chatbot = tf.keras.models.load_model(
    "model_checkpoint_cb.h5",
    custom_objects={
            "PositionalEncoding": PositionalEncoding,
            "MultiHeadAttentionLayer": MultiHeadAttentionLayer,
        },
    compile=False
)

sentences = [
    "Hello, my name is Omid . what about you?",
    "how was your day",
    "How old are you?",
    "where have you been",
    "do you like pizza?",
    "my favourite color is blue",
    None, None, None
]

In [14]:
evaluate(dataset_hp, chatbot, tokenizer, sentences)

-evaluating ...

Input: Hello, my name is Omid . what about you?
Output: i am not sure working , i guess . . .

Input: how was your day
Output: i am sorry , honey . i am not going to need some time .

Input: How old are you?
Output: i do not know . i am just having some people you to say children need a new ones . but at least they do with me , do with me , do the middle of the teens .

Input: where have you been
Output: i am not going anywhere . i am here . i am right here . i will not let s get you .

Input: do you like pizza?
Output: i am not sure . i am betting on you .

Input: my favourite color is blue
Output: i am not going anywhere . i have got to do not like you .

Input: i am not going anywhere . i have got to do not like you .
Output: i am not going anywhere . i am right here . i win .

Input: i am not going anywhere . i am right here . i win .
Output: i am not going to cover for this . it change .

Input: i am not going to cover for this . it change .
Output: i am not going